# Exercise 1: Simple Inference

In this exercise, you will be running inference on a few images.
The primary goal of the first exercise is to familiarize you with the workflow for inference.

You will be creating a vehicle detection application where the model counts how many vehicles are found in an image. The image you will use is:

<img src="cars_1900_first_frame.jpg">

There appears to be 9 vehicles in the image. Let's see how the computer vision models do.

### Important! The quiz will ask you how many vehicles were detected in the last step.


## Step 1: Running on the DevCloud

In the exercises of this tutorial including this one, we will be taking a look at running inferences on a variety of devices like VPU and FPGA. 
In order to access these devices, we will be taking advatnage of the Intel DevCloud for Edge. 
This is a free cloud service from Intel Corporation where users get to try some of the newest edge devices from Intel. 
For more information as well as instructions to sign up for an account, visit [devcloud.intel.com](https://devcloud.intel.com). 
For the exercises in this tutorial you do not need to sign up for an accout on the DevCloud. 
We will just be sending your submission to the cloud to run inference.

To run a workload on the DevCloud, we must submit a **job** to a job queue.
A job consists of a bash script to run the workload, along with any auxillary file used in the workload.
When submitting a job to a cloud, you can request a certail hardware resource such as VPU or FPGA. 
Then these files are sent over to the DevCloud, the bash script is run, and finally the result is made available to us.

For this first step, we'll start by running a simple workload on the DevCloud. 
Using the `model_downloader.py`, let's get the list of available models for inference.
The `model_downloader.py` is already available on the DevCloud, so for this we just need to create the bash script.
In jupyter, cells beginning with `%%writefile myfile.txt` will dump the contents of the cell into a file, named `myfile.txt` in this case.
So the next code cell creates a file `show_all_models.sh`.

`show_all_models.sh` is currently set up to print the help menu of the `model_downloader.py`.
Modify the following cell so that it displays the list of all the available models instead.

In [ ]:
%%writefile show_all_models.sh
/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --help

To submit this to the DevCloud we need to make a REST API call. 
However this part is beyond the scope of this tutorial.
So we have provided the utility function `submitToDevCloud()` for submitting your job.
This function will submit your job, and (by default) print the stdout and stderr.

This function has two required arguments, and one optional argument.
- script: String. Relative path to the bash script to run. Required.
- node_type: String. Selects the node to request. Must be one of CPU, GPU, VPU, FPGA. Required.
- script_args: List of strings. Arguments to pass to the bash script. Optional. Default is empty array.
- files: List of strings. Specifies the auxillary files. Optional. Default is empty array.

Run the following cell to submit `show_all_models.sh` to the job queue.

In [ ]:
from utils import submitToDevCloud
submitToDevCloud("show_all_models.sh", "CPU")

That's it! You have just ran a workload on the DevCloud.

## Step 2: Running Inference

Next step is running the inference itself.
This section will make use of the Inference Engine that we have covered. 

To run the workload on the DevCloud, we need to create two files. 
We a python script that will run the iunference using Inference Engine.
Additionally, we need a shell script that will start this python file.
Let's start with the python file, which we call `main.py`.

The instructions for completing the file is broken into steps. 
In the cell, the parts that need to be modified is signified by `##! ... !##`
The number in parenthesis shows the step in the instruction that this corresponds to.

**You can click on the steps to get detailed instructions.**
Follow the instructions to complete `main.py`. 
If you get stuck on any of the stps, refer to the slide deck from course 1 video 6.

<details>
    <summary><b>(2.1)</b> Complete the <code>prepImage()</code> function by finding the NCHW values from the network.</summary>
    
The input image will be loaded using OpenCV, so several image processing steps needs to be done.
First, it will have the wrong size. so the image must be reshaped using `cv2.resize()` function.
Second, OpenCV loads an image in HWC format whereas the network expects an NCHW format. 
So the image must first be transposed using `transpose()` method of numpy arrays. 
Then the N dimention must be added using the `reshape()` method.

As the preprocesisng is outside of the toolkit, they are already implemented for you.
However it is missing the sizes for dimensions NCHW of the network input.

Complete the `prepImage()` function by getting the values for `n`, `c`, `h` and `w` from the function input `net`.

</details><br/>

<details>
    <summary><b>(2.2)</b> Create IENetwork models for <code>vehicle-detection-adas-0002</code>. Use the provided paths for the models.</summary>
    
The model `vehicle-detection-adas-0002` has already been downloaded for you. 
The constructor for `IENetwork` object takes the path to the two files as input.

</details><br/>

<details>
    <summary><b>(2.3)</b> Preprocess the image with <code>prepImage()</code>.</summary>
    
Prepare the image for inference using the prepImage() function from earlier. Remember that this function outputs a modiified image, instead of doing the modification in place.

</details><br/>

<details>
    <summary><b>(2.4)</b>  Create IECore and ExecutableNetwork objects.</summary>
    
Remember that you need to create IECore object first.

ExecutableNetwork is created from the load_network() method of IECore object. The IENetwork should be the one created earlier, and the device should be the one in `device` variable. This variable is set by the commandline input to the main.py script.

</details><br/>

<details>
    <summary><b>(2.5)</b>  Run synchronous inference.</summary>
    
Synchronous, or blocking, run is started with `infer()` method. Remember that for the `inputs` argument, you will need the name of the input layer so that you can input the image.

</details><br/>

<details>
    <summary><b>(2.6)</b>  Get the output array and run <code>printCount()</code> to see how many vehicles were detected.</summary>
    
Remmber that the output is a dictionary, with output layer name as key and the result array as value. You will need the name of the output layer to get the output array.

`printCount()` takes the output array and counts the number of vehicle locations with confidence greater than the `prob_threshold` argument (default 50%). 

A quick explanation of what `prinCount` does. `vehicle-detection-adas-0002` returns 100 potential regions where an object might be.
For every potential object, the model assigns a probability that it is an object.
So to find the vehicles in the image, you need to look for entries over a certain threshold probability.
The function simply loops over the potential regions and counts the number of regions with confidence values greater than the `prob_threshold`.

The model also provides bounding boxes for where the potential object is, and it returns an index to the detected object. 
Though not shown in this example, this information can be processed and placed on the original image.

</details><br/>


In [ ]:
%%writefile main.py
import os
import sys
from openvino.inference_engine import IECore, IENetwork
import cv2

# Prepares image for imference
# inputs:
#     orig_image - numpy array containing the original, unprocessed image
#     net        - IENetwork object
# output: 
#     preprocessed image.
def prepImage(orig_image, net):
    
    ##! (2.1) Find n, c, h, w from net !##

    input_image = cv2.resize(orig_image, (w, h))
    input_image = input_image.transpose((2, 0, 1))
    input_image.reshape((n, c, h, w))
    return input_image

# Processes the result. Prints the number of detected vehices.
# inputs:
#    detected_obects - numpy array containing the ooutput of the model
#    prob_threashold - Required probability for "detection"
# output:
#    numpy array of image wtth rectangles drawn around the vehicles.
def printCount(detected_objects, prob_threshold=0.5):
    detected_count = 0
    for obj in detected_objects[0][0]:
        # Draw only objects when probability more than specified threshold
        if obj[2] > prob_threshold:
            detected_count+=1    
    print("{} vehicles detected.".format(detected_count))

# Getting the device as commandline argument
device = sys.argv[1]
    
##! (2.2) create IENetwork object for vehicle-detection-adas-0002 !##
xml_path="/data/intel/vehicle-detection-adas-0002/FP16-INT8/vehicle-detection-adas-0002.xml"
bin_path="/data/intel/vehicle-detection-adas-0002/FP16-INT8/vehicle-detection-adas-0002.bin"


image_path = "cars_1900_first_frame.jpg"
original_image = cv2.imread(image_path)

##! (2.3) Preprocess the image. !##

##! (2.4) Create IECore and ExecutableNetwork object. Use the device variable for targetted device !##

##! (2.5) Run synchronous inference. !##

##! (2.6) Run printCount. Make sure you extracted the array result form the dictionary returned by infer(). !##



Now that you have the python script, we need the bash script.
This script is provided for you.
Run the following cell to create `run.sh`.

In [ ]:
%%writefile run.sh
DEVICE=$1
source /opt/intel/openvino/bin/setupvars.sh
python3 main.py $DEVICE

One important note here is that the bash script takes in one argument (`$1` is the first positional argument in bash). 
This input is then passed on to the python script so that you can set the device argument.
Inputs to the script is passed in using the `script_args` argument.

Run the following cell to run the job on the CPU.

In [ ]:
from utils import submitToDevCloud
submitToDevCloud("run.sh", "CPU", script_args=["CPU"], files=["cars_1900_first_frame.jpg","main.py"])

Now run the following cell to run the job on the GPU. Note that this one will take noticeably longer. There is some on-the-fly compiling of OpenCL for GPU, which takes some time.

In [ ]:
from utils import submitToDevCloud
submitToDevCloud("run.sh", "CPU", script_args=["GPU"], files=["cars_1900_first_frame.jpg","main.py"])


Congratulations! You have successfully run the inference workload on a GPU and a CPU. 

## Step 4: Quiz question

For the final step, let's try lowering the required confidence to 0.01 (e.g. 1%) and see how many vehicles are detected by the model. 
You will have to go back to the `main.py` cell, and add `prob_threshold` argument to `printCount` function.
**The quiz will ask you how many vehicles were detected by the `vehicle-detection-adas-0002` with this setting.**